In [11]:
%%writefile head.py
# =---------------------------------------------------------------------------=
# HEAD: Homo Entropicus Agents, Dude
# Version: 0.1.0
# Author: Felippe Alves
# E-Mail: flipgm@gmail.com
# =---------------------------------------------------------------------------=
# This module contains the basic dynamics generator, the class for Entropic
# Agents Societies (and the particular case of Teacher-Student learning
# scenario), and some utility funtions used in the analysis of such
# systems.
#
# I separated it in some sections:
#  1. Simulation Interface
#    1.1 Utility functions
#    1.2 Dynamics Interface
#  2. Agents Society Model
#    2.1 Theory
#    2.2 Model
#  3. Teacher-Student Scenario
#  4. Restricted Universe of Discorse Scenario
#  5. Analysis Interface
#
# Everything is pretty straight forward, and should be no problem to understand
# what the code does from reading it.

import functools
from itertools import count , islice
from copy import deepcopy
from collections import defaultdict
from numpy import (cos, sin, sqrt, r_, ones, tile, vstack, array, ndindex, ndim,
                   log, outer, sign, eye, infty as oo, einsum, zeros, stack,
                   zeros_like, fill_diagonal, clip, pi, split, arange, empty, NaN,
                   linspace)
from numpy.random import choice, randn, rand
from numpy.linalg import norm, qr
from scipy.stats.distributions import norm as normal

__all__ = ['triangle_indices', 'rand_sphere', 'random_at_angle',
           'dynamics', 'n_steps', 'record_trajectory',
           'BOMAgentSociety', 'random_initialization',
           'TeacherStutdentScneario', 'RUDBOMAgentSociety']

# # Simulation Interface

# ## Utility functions
Phi = normal.cdf
G = normal.pdf
row_norm = functools.partial(norm, keepdims=True, axis=1)


def triangle_indices(N):
    """Compute all triples of integer indices and, as consequence,
    all the closed oriented triangles in a complete graph with N
    vertices.
    Input:
    -----
      N: int - the number of vertices in a complete graph

    Output:
    -------
      N(N-1)(N-2) x 3 array where each line is a triple of
      indices in a triangle
    """
    tri_idx = array([(a, b, c) for
                     (a, b, c) in ndindex((N, N, N))
                     if (a!=b and a!=c and b!=c)])
    return tri_idx


def rand_sphere(K, size=1):
    """Computes K-dimensional vectors distributed uniformly
    in the unit sphere.
    Input:
    ------
      K: int - the dimension of the vector space
      size (default 1): int - the number of vectors to sample
    Output:
    -------
      array with shape (K,) if size==1 or (size, K) if size>1
    """
    v = randn(size, K)
    v /= row_norm(v)
    if size == 1:
        v = v.reshape(K)
    return v


def random_at_angle(B, theta):
    """Computes a random vector with angle theta from a vector B.
    Input:
    ------
      B: 1d array - the reference vector
      theta: float - the angle from the reference vector B
    """
    K = B.shape[0]
    B0, v = (qr(vstack([B, randn(K)]).T)[0]*r_[-1, 1]).T
    w = cos(theta)*B0 + sin(theta)*v
    w /= norm(w)
    w *= sign(cos(theta)*w@B0)
    return w


def random_from_cosine(B, cosine):
    """Computes a random vector with a given cosine with the vector B.
    Input:
    ------
      B: 1d array - the reference vector
      cosine: float - the cosine with the reference vector B
    """
    K = B.shape[0]
    B0, v = (qr(vstack([B, randn(K)]).T)[0]).T
    B0 *= sign(B0@B)
    w = cosine*B0 + sin(arccos(cosine))*v
    assert abs(norm(w)-1) < 1e-5
    return w


## Dynamics interface
def dynamics(system, *, in_place=True, **params):
    """Generator to loop over systems (obejcts) with
    a update and a get_state methods.
    Input:
    ------
      system: any object with the following interface:
        system.update(**params) - a method for updating the system state
        system.get_state(**params) - a method to retrieve the system state
      in_place (default False): bool - if True, the updates are done to the
      system passed in the call. If false, a deepcopy is made before looping
      **params: any other parameter to be passed to the system methods
    Output:
    -------
      an iterator over the system dynamics
    """
    if not in_place:
        system = deepcopy(system)
    for _ in count():
        yield system.get_state(**params)
        system.update(**params)


def n_steps(infinite_iterator, n):
    """An alias to enumerate(itertools.islice(ifinite_iteretor, n))"""
    return enumerate(islice(infinite_iterator, n))


def record_trajectory(system, number_of_steps, *,
                      record_step=lambda n: True, **params):
    """A simple materialization of the dynamics iterator for a finite
    number of steps, possibly filtered.
    Input:
    ------
      system: object following the dynamics interface
      (see sas.dyamics docstring)
      number_of_steps: int - the number of steps to iterate the dynamics
      record_step: callable accepting an integer argument - funtion to
      decide when to record a given interger step
      **params: all other parameters to pass to system's methods
    Output:
    -------
      array of type(system.get_state()) with shape (number_of_steps,)
    """
    trajectory = [s for (n, s) in n_steps(dynamics(system, **params),
                                          number_of_steps)
                  if record_step(n)]
    return array(trajectory)

# # Agents Society Model

# ## Theory

def Z(phi_w, phi_mu):
    """Evidence for the Bayesian Opinion-Trust learning model
    Input:
    ------
      phi_mu, phi_w: floats in [0,1] range"""
    return phi_mu + phi_w - 2*phi_mu*phi_w


def derivatives_lnZ(arg, const, z):
    """Compute the value of the first and second derivatives of ln(z) for a given `arg`, `const` and value of `z`
    Input:
    ------
      arg: float - the argument in the derivative
      const: float - the constant argument in the derivative
      z: float - the value of Z(arg, const)
    """
    dlnZ_darg =  (1-2*const)*G(arg)/z
    d2lnZ_darg = -dlnZ_darg*(dlnZ_darg + arg)
    return dlnZ_darg, d2lnZ_darg


def compute_deltas_w_and_C(w, C, x, sigma, phi_mu_l, z):
    gamma = sqrt(x @ (C @ x)) / norm(x)
    hs_g = w @ x * sigma / gamma 
    dlnZ_dhs_g, d2lnZ_dhs_g = derivatives_lnZ(hs_g, phi, z)
    dw = dlnZ_dhs_g*(C@x)*sigma/gamma
    dC = d2lnZ_dhs_g*C@outer(x, x)@C/(gamma*gamma)
    return [dw, dC]


def compute_deltas_mu_and_s2(mu, s2, phi_hs_g, z):
    lmbda = sqrt(1+s2)
    mu_l = mu/lmbda
    dlnZ_dmu_l, d2lnZ_dmu_l = derivatives_lnZ(mu_l, phi_w, z)
    dmu = dlnZ_dmu_l*s2/lmbda
    ds2 = d2lnZ_dmu_l*s2*s2/(1+s2)
    return [dmu, ds2]


def compute_deltas(w, C, mu, s2, x, sigma):
    """Compute the update values for w, C, mu and s2 given the
    example (x,sigma). The values correspond to the Bayesian
    Opinion-Trust model optimal updates.
    Input:
    ------
      w: array with shape (K,) - agent's opinion vector
      C: array with shape (K, K) [positive definite] - agent's opinion
      uncertainty
      mu: float - agent's distrust value
      s2: float [greater than 0] - agent's distrust uncertainty
    Output:
    -------
      list with the update values for w, C, mu and s2
    """
    K = w.shape[0]
    h = x@w
    gamma = sqrt(x@C@x)/norm(x)
    lmbda = sqrt(1+s2)
    hs_g = h*sigma/gamma
    mu_l = mu/lmbda
    phi_w = Phi(hs_g)
    phi_mu = Phi(mu_l)
    z = Z(phi_w, phi_mu)
    dlnZ_dmu_l, d2lnZ_dmu_l = derivatives_lnZ(mu_l, phi_w, z)
    dlnZ_dhs_g, d2lnZ_dhs_g = derivatives_lnZ(hs_g, phi_mu, z)
    delta_mu = dlnZ_dmu_l*s2/lmbda
    delta_s2 = d2lnZ_dmu_l*s2*s2/(1+s2)
    delta_w = dlnZ_dhs_g*sigma*C@x/gamma
    delta_C = d2lnZ_dhs_g*C@outer(x, x)@C/(gamma*gamma)
    return [delta_w, delta_C, delta_mu, delta_s2]


def learning_cost(w, C, mu, s2, x, sigma):
    """Compute the learning cost for w, C, mu and s2 given the
    example (x,sigma). The values correspond to the Bayesian
    Opinion-Trust model learning cost.
    Input:
    ------
      w: array with shape (K,) - agent's opinion vector
      C: array with shape (K, K) [positive definite] - agent's opinion
      uncertainty
      mu: float - agent's distrust value
      s2: float [greater than 0] - agent's distrust uncertainty
    Output:
    -------
      float value of learning cost
    """
    h = x@w
    gamma = sqrt(x@C@x)/norm(x)
    lmbda = sqrt(1+s2)
    phi_w = Phi(h*sigma/gamma)
    phi_mu = Phi(mu/lmbda)
    return -log(Z(phi_mu, phi_w))


# ## Model
class HEODAgentSociety(object):  
    def __init__(self, w0, C0, mu0, s20, *args, **kwargs):
        """Society of "Homo Entropicus" Opinion-Distrust agents.
        Input:
        ------
          w0: array with shape (N, K) - opinion vector with dimension K for
          N agents - representing the opinion weight vector;
          C0: array with shape (N, K, K) - opinion uncertainty for each 
          agent - representing the opinion uncertainty; 
          mu0: array with shape (N, N) - distrust array for each agent
          - representing the distrust proxy;
          s20: array with shape (N, N) - distrust uncertainty for each
          agent - reprsenting the distrust uncertainty.
        """
        self.w = w0.copy()
        self.N, self.K = w0.shape
        assert C0.shape == (self.N, self.K, self.K)
        self.C = C0.copy()
        assert mu0.shape == s20.shape == (self.N, self.N)
        self.mu = mu0.copy()
        self.s2 = s20.copy()
        self.initial_state = [w0.copy(), C0.copy(), mu0.copy(), s20.copy()]
        self._state_struct = [('w', 'f8', self.K),
                              ('C', 'f8', (self.K, self.K)),
                              ('mu', 'f8', self.N),
                              ('s2', 'f8', self.N)]
        self.interaction_counter = 0
        
    @classmethod
    def random(cls, N, K, C0, s20, *args, **kwargs):
        """Provides a simple and random set of initial values for BOTAgentSociety.
        Input:
        ------
        N: int - the number of agents
        K: int - the agents internal dimension
        
        Output:
        -------
        list with initial values for w, C, mu, and s2"""
        w0 = randn(N, K)
        w0 /= row_norm(w0)
        C0 = tile(C0*eye(K)/K, (N, 1, 1))
        mu0 = randn(N, N)
        fill_diagonal(mu0, -100)
        s20 = s20*ones((N, N))
        return cls(w0, C0, mu0, s20, *args, **kwargs)

    def agent_answer(self, i, x, *, real_epsilon=0.0, **params):
        sigma = sign(self.w[i]@x)*choice([-1, 1], p=[real_epsilon,
                                                     1-real_epsilon])
        return sigma

    def learning_amplitude(self, i, j, x, *args, constants=(), scales=(1.,1.,1.,1.), **params):
        wi, Ci, mui, s2i = self[i]
        sigma_j = self.agent_answer(j, x, **params)
        Dw, DC, Dmuij, Ds2ij = compute_deltas(wi, Ci, mui[j], s2i[j], x, sigma_j)
        sw, sC, smuij, ss2ij = scales
        if 'C' in constants:
            DC[:] = 0.0
        if 's2' in constants:
            Ds2ij = 0.0
        return [Dw/sw, DC/sC, (j, Dmuij/smuij, Ds2ij/ss2ij)]

    def move_agent(self, i, deltas, *args, constants=(), **params):
        self[i] = deltas
        if 'norm' in constants:
            self.normalize_agent_opinion(i, *args, **params)
        if 'bounds' in constants:
            self.normalize_agent_distrust(i, *args, **params)

    def interaction(self, i, j, x, *args, symmetric=False, **params):
        Deltas_i = self.learning_amplitude(i, j, x, *args, **params)
        if symmetric:
            Deltas_j = self.learning_amplitude(j, i, x, *args, **params)
            self.move_agent(j, Deltas_j, *args, **params)
        self.move_agent(i, Deltas_i, *args, **params)
        self.interaction_counter += 1

    def get_state(self, *args, **kwargs):
        state = zeros(self.N, self._state_struct)
        for n in state.dtype.names:
            state[n] = getattr(self, n)[:]
        return state

    def normalize_agent_opinion(self, i, *, opinion_norm=1., **params):
        self.w[i] *= opinion_norm/norm(self.w[i])

    def normalize_agent_distrust(self, i, *, distrust_bound=1., **params):
        clip(self.mu[i], -distrust_bound, distrust_bound, self.mu[i])

    def __getitem__(self, i):
        """Just a convinience to access agents properties"""
        values = self.w[i], self.C[i], self.mu[i], self.s2[i]
        return values

    def update(self, *args, **params):
        x = rand_sphere(self.K)
        i, j = choice(self, size=2, replace=False)
        self.interaction(0, 1, x, *args, **params)

    def __setitem__(self, i, deltas):
        """Conviniece to set agents properties values"""
        Dw, DC, (j, Dmu, Ds2) = deltas
        self.w[i] += Dw[:]
        self.C[i] += DC[:]
        self.mu[i, j] += Dmu
        self.s2[i, j] += Ds2

    def reset(self):
        """Convinience to restore the initial state"""
        w0, C0, mu0, s20 = self._initial_state
        self.w = w0.copy()
        self.C = C0.copy()
        self.mu = mu0.copy()
        self.s2 = s20.copy()
        self.interaction_counter = 0

## Teacher-Student Scenario
class TeacherStutdentScneario(HEODAgentSociety):
    def __init__(self, teacher, theta_0, C0, mu0, s20, *args, **kwargs):
        """Teacher/Student learning scenario
        Input:
        ------
          teacher: array - the teacher vector
          theta_0: flaot - initial angle between teacher and student
          C0 - 2d array - initial student opinion uncertainty
          mu0, s20: flaot, float > 0 - intial distrust and respective
        uncertainty for the student.
        """
        assert ndim(teacher) == 1
        student = random_at_angle(teacher, theta_0)
        w0 = vstack([student, teacher])
        K = teacher.shape[0]
        w0 /= row_norm(w0)
        super().__init__(w0, C0, mu0, s20, *args, **kwargs)

    def update(self, *args, **params):
        x = rand_sphere(self.K)
        self.interaction(0, 1, x, *args, **params)


## Restricted Universe of Discorse Scenario
class RUDHEODAgentSociety(HEODAgentSociety):
    """Restricted Universe of Discourse for the "Homo Entropicus" Opinion-Distrust
    Agent Society."""
    def pick_issue(self, *args, issue_list=None, **params):
        if issue_list is not None:
            k = choice(len(issue_list))
            x = issue_list[k]
        else:
            x = rand_sphere(K)
        return x

    def pick_agents(self, *args, **params):
        i, j = choice(self.N, size=2, replace=False)
        return i, j

    def update(self, *args, **params):
        x = self.pick_issue(*args, **params)
        i, j = self.pick_agents(*args, **params)
        self.interaction(i, j, x, *args, **params)


## Analysis Interface
class HEODAgentSocietyTrajectory(object):
    _observables = 'overlap distrust trust balance mean_balance frustration mean_frustration coherence coherence_mean'.split()

    def __init__(self, trajectory_array):
        self.w = trajectory_array['w']
        self.C = trajectory_array['C']
        self.mu = trajectory_array['mu']
        self.s2 = trajectory_array['s2']
        self.T, self.N, self.K = self.w.shape
        self._trajectory_array = trajectory_array

    @property
    def triangle_indices(self):
        if not hasattr(self, '_triangle_indices'):
            self._triangle_indices = triangle_indices(self.N)
        return self._triangle_indices.T

    @property
    def normalized_w(self):
        return self.w/norm(self.w, axis=2, keepdims=True)

    @property
    def overlap(self):
        """Compute the normalized overlap between agents"""
        u = self.normalized_w #self.w/norm(self.w, axis=2, keepdims=True)
        return einsum('...ij,...kj->...ik', u, u)

    @property
    def distrust(self):
        """Compute the distrust estimate"""
        return Phi(self.mu/sqrt(1+self.s2))

    @property
    def trust(self):
        """Compute the trust as 1 - 2*distrust"""
        return 1-2*self.distrust

    @property
    def balance_naive(self):
        t = self.trust
        return einsum('...ij,...jk,...ik->...ijk', t, t, t)

    @property
    def balance(self):
        I, J, K  = self.triangle_indices
        return self.balance_naive[:, I, J, K]

    @property
    def mean_balance(self):
        return self.balance.mean(axis=1)

    @property
    def frustration_naive(self):
        o = self.overlap
        return einsum('...ij,...jk,...ik->...ijk', o, o, o)

    @property
    def frustration(self):
        I, J, K = self.triangle_indices
        return self.frustration_naive[:, I, J, K]

    @property
    def mean_frustration(self):
        return self.frustration.mean(axis=1)

    @property
    def coherence_naive(self):
        c = (self.overlap * self.trust)
        return einsum('...ij,...jk,...ik->...ijk', c, c, c)

    @property
    def coherence(self):
        I, J, K = self.triangle_indices
        return self.coherence_naive[:, I, J, K]

    @property
    def coherence_mean(self):
        return self.coherence.mean(axis=1)

    @property
    def observables(self):
        """Creates the dict of observables"""
        return {n: getattr(self, n) for n in self._observables}


Writing head.py


In [12]:
%%writefile judges.py
class JudgePanels(HEODAgentSociety):
    N, K, P = 18, 5, 14  # number of agents, internal dimension and number of issues
    panel_compositions = ('Aaa', 'Aab', 'Abb', 'Bbb', 'Bba', 'Baa')  # types of panel by political affiliation
    panel_indices = vstack(split(arange(N), len(panel_compositions)))  # agent indices for each panel
    panel_map = {c: i for (c, i) in zip(panel_compositions, panel_indices)}
    focal_agents = list(range(0,18,3))  # the agent indices corresponding to the capital letters
    law_vector = ones(K)/norm(ones(K))  # vector repesenting the kwoledge of the law
    party_vector = r_[-1.,-1.,0.,1.,1.]  # vector representing the political attitude
    party_vector /= norm(party_vector) 
    interaction_counter = 0
    
    def __init__(self, alpha_law, alpha_party, alpha_person, mu0XX, mu0XY, c0A, c0B, s20, *args, **params):
        """Society of agents representing panels of judges apointed by presidents 
        of political affiliation A or B.
          The society has 18 agents interacting in groups of 3, representing the 6 
        possible composition panels: Aaa, Aab, Abb, Bbb, Bba, Baa.
          The notation Xyz is to be understood as the focal agent has political
        affiliation X in a panel with agents with political affiliations y and z.
          The initial opinion state for the judges is composed by a law component,
        representing the knowledge of the law and common to all agents, a party
        component, represent the political attitude and opposite for A and B, and
        a personality component of random charater.
          To initialize the society, the weights `alpha_law`, `alpha_party` and 
        `alpha_person` are used to build the opinion vectors.
          The opinion uncertainty is a party dependent multiple of the identity 
        matrix, with the multiples being the parameters `c0A` and `c0B`.
          For the distrust, we use two paramenters `mu0XX` and `mu0XY` to characterize
        the distrust attributed between members of party X to members of X and Y,
        respectively.
          Last, the distrust uncertainty is equal to everyone, being a multiple of the
        matrix will all entries equal to 1 and given by the parameter `s20`.
        
        Inputs:
        =======
          - alpha_law, alpha_party, alpha_person: floats - weights to initial opinion
          vector components of law, party and personality.
          - mu0XX, mu0XY: floats - intra and extra-party initial distrust
          - c0A, c0B: positive floats - initial opinion uncertainties for each party
          - s20: positive float - initial distrust uncertainty
          
        API:
        ====
          Although you can use any method inside this object, it intended to be used
          a iterator over the dynamics, so we only give a description for the 
          methods provided to this end:
          - update(*args, **param): Method to make a random move using the Entropic
          Dynamics of Learning. The Options section covers the parameters recognized
          - reset(): Method to reset the system to its initial state.
          - observables: Propery to compute and return all the observables in this
          system. Notice that observable denotes a quantity of interest, not anything
          readable in memory.
          
          Options:
          --------
          opinion_norm: positive float; default is 1.0 - gives the norm for the agents' 
          opinion vectors.
          distrust_bound: positive float; defautl is 1.0 - gives the bound for the
          interval containing the distrust proxies of the agents.
          constants: tuple with acceptable values 'norm', 'C', 's2', 'bounds'; default 
          is ('C', 'norm') - each value
          keeps the respective variable fixed under the dynamics, where 'norm' is for the 
          opinion vectors, 'C' is for the opinion uncertainty, 's2' is for the distrust
          uncertainty and 'bounds' is for the distrust interval.
        """
        assert (c0A >= 0) and (c0B >= 0) and (s20 >= 0)
        self.alpha_law = alpha_law
        self.alpha_party = alpha_party
        self.alpha_person = alpha_person
        self.mu0XX, self.mu0XY = mu0XX, mu0XY
        self.c0A, c0B = c0A, c0B
        self.s20 = s20
        
        # Building the initial opinion vectors
        #             law component                 party compoenent
        A = alpha_law*self.law_vector + alpha_party*self.party_vector
        B = alpha_law*self.law_vector - alpha_party*self.party_vector
        #            A a a A a b A b b B b b B b a B a a                  personality component
        w0 = vstack([A,A,A,A,A,B,A,B,B,B,B,B,B,B,A,B,A,A]) + alpha_person*randn(self.N,self.K)
        w0 /= row_norm(w0)
        
        # Building the initial opinion uncertainties
        I = eye(self.K)
        CA, CB = c0A*I, c0B*I
        #            A  a  a  A  a  b  A  b  b  B  b  b  B  b  a  B  a  a
        C0 = stack([CA,CA,CA,CA,CA,CB,CA,CB,CB,CB,CB,CB,CB,CB,CA,CB,CA,CA], axis=0)
        
        # Building the initial distrust proxies - X in (A,B), Y in (A,B), i, j, k are agents
        #                      i     j     k
        muXxx_tile = array([[-10, mu0XX, mu0XX],   # ii==XX, ij==XX, ik==XX
                            [mu0XX, -10, mu0XX],   # ji==XX, jj==XX, jk==XX
                            [mu0XX, mu0XX, -10]])  # ki==XX, kj==XX, kk==XX
        muXxy_tile = array([[-10, mu0XX, mu0XY],   # ii==XX, ij==XX, ik==XY
                            [mu0XX, -10, mu0XY],   # ji==XX, jj==XX, jk==XY
                            [mu0XY, mu0XY, -10]])  # ki==XY, kj==XY, kk==XX
        muXyy_tile = array([[-10, mu0XY, mu0XY],   # ii==XX, ij==XY, ik==XY
                            [mu0XY, -10, mu0XX],   # ji==XY, jj==XX, jk==XX
                            [mu0XY, mu0XX, -10]])  # ki==XY, kj==XX, kk==XX
        mu_map = {c: mt for (c, mt) in zip(self.panel_compositions,[muXxx_tile, muXxy_tile, muXyy_tile]*2)}
        mu0 = zeros((self.N, self.N))
        panel_pairs = stack([vstack([[i,j] for i in p for j in p]) for p in self.panel_indices], axis=0)
        for panel, indices in self.panel_map.items():
            pair_indices = vstack([[i,j] for i in indices for j in indices])
            I, J = split(pair_indices, 2, axis=1)
            mu0[I, J] = mu_map[panel].reshape(I.shape)
            
        # Building the initial distrust uncertainty
        s20 = s20*ones((self.N, self.N))
        fill_diagonal(s20, 0.0)
        
        # let the super class prepare the remaining of the object
        super().__init__(w0, C0, mu0, s20)
        
    @property
    def issue_list(self):
        if not hasattr(self, '_issue_list'):
            thetas = linspace(-pi/2,pi/2, self.P)
            self._issue_list = vstack([cos(t)*self.law_vector + sin(t)*self.party_vector
                                        for t in thetas])
        return self._issue_list
    
    def pick_issue(self, *args, **params):
        k = choice(len(self.issue_list), size=6, replace=True)
        x = self.issue_list[k]
        return x

    def pick_agents(self, *args, **params):
        pairs = vstack([choice(p, size=2, replace=False)
                        for p in self.panel_indices])
        return pairs

    def update(self, *args, constants=('norm', 'C'), **params): 
        xs = self.pick_issue(*args, **params)
        pairs = self.pick_agents(*args, **params)
        for ((i,j),x) in zip(pairs, xs):
            self.interaction(i, j, x, *args, **params)
            
    @property
    def votes(self):
        if not hasattr(self, '_votes'):     
            h = self.w[self.focal_agents, :]@self.issue_list.T
            self._votes = (self.panel_compositions, h/row_norm(h))
        return self._votes
    
    @property
    def panel_overlaps(self):
        if not hasattr(self, '_panel_overlaps'):
            panels, votes = self.votes
            overlaps = votes@votes.T
            panel_pairs = array([[' '.join([p1,p2]) for p1 in panels] for p2 in panels])
            self._panel_overlaps = (panel_pairs, overlaps)
        return self._panel_overlaps
        
    @property
    def observables(self):
        if not hasattr(self, '_observables'):
            self._observables = 'votes panel_overlaps'.split()
        return {n:getattr(self, n) for n in self._observables}

Writing judges.py


In [3]:
judges = JudgePanels(1., 1., 1., -2., -1., 2., 4., .5)

In [4]:
obs = judges.observables

In [5]:
obs['votes']

(('Aaa', 'Aab', 'Abb', 'Bbb', 'Bba', 'Baa'),
 array([[ 0.00799277,  0.10160535,  0.189313  ,  0.26601846,  0.3272639 ,
          0.36948995,  0.39024259,  0.38831574,  0.3638214 ,  0.31818308,
          0.25405311,  0.17515851,  0.08608432, -0.00799277],
        [ 0.16368149,  0.2428335 ,  0.30787291,  0.35501986,  0.38153435,
          0.38587545,  0.36779087,  0.32833163,  0.26979094,  0.19557098,
          0.10998515,  0.01800739, -0.07501691, -0.16368149],
        [-0.22295679, -0.1421403 , -0.05306314,  0.03909787,  0.12898664,
          0.21137918,  0.28148713,  0.33523608,  0.36950232,  0.38229442,
          0.37286897,  0.34177373,  0.29081584,  0.22295679],
        [ 0.35378336,  0.36673973,  0.35838251,  0.32919741,  0.28088055,
          0.21623994,  0.13903224,  0.0537445 , -0.03466668, -0.12106316,
         -0.20042388, -0.2681367 , -0.32026639, -0.35378336],
        [-0.03039233,  0.06403244,  0.15473587,  0.23644662,  0.30441594,
          0.35469372,  0.38435799,  0.391

In [6]:
obs['panel_overlaps']

(array([['Aaa Aaa', 'Aab Aaa', 'Abb Aaa', 'Bbb Aaa', 'Bba Aaa', 'Baa Aaa'],
        ['Aaa Aab', 'Aab Aab', 'Abb Aab', 'Bbb Aab', 'Bba Aab', 'Baa Aab'],
        ['Aaa Abb', 'Aab Abb', 'Abb Abb', 'Bbb Abb', 'Bba Abb', 'Baa Abb'],
        ['Aaa Bbb', 'Aab Bbb', 'Abb Bbb', 'Bbb Bbb', 'Bba Bbb', 'Baa Bbb'],
        ['Aaa Bba', 'Aab Bba', 'Abb Bba', 'Bbb Bba', 'Bba Bba', 'Baa Bba'],
        ['Aaa Baa', 'Aab Baa', 'Abb Baa', 'Bbb Baa', 'Bba Baa', 'Baa Baa']],
       dtype='<U7'),
 array([[ 1.        ,  0.90350095,  0.77839003,  0.2686965 ,  0.99446947,
          0.84250242],
        [ 0.90350095,  1.        ,  0.43421794,  0.65559232,  0.85349143,
          0.99207787],
        [ 0.77839003,  0.43421794,  1.        , -0.39554359,  0.84001846,
          0.31761454],
        [ 0.2686965 ,  0.65559232, -0.39554359,  1.        ,  0.16604679,
          0.74525953],
        [ 0.99446947,  0.85349143,  0.84001846,  0.16604679,  1.        ,
          0.7812662 ],
        [ 0.84250242,  0.99207787,  0

In [7]:
trj = record_trajectory(judges, 30, record_step=lambda n: n==30-1)

In [8]:
trj[0]['w'].shape

(18, 5)

In [9]:
obsf = judges.observables

In [10]:
obsf['panel_overlaps']

(array([['Aaa Aaa', 'Aab Aaa', 'Abb Aaa', 'Bbb Aaa', 'Bba Aaa', 'Baa Aaa'],
        ['Aaa Aab', 'Aab Aab', 'Abb Aab', 'Bbb Aab', 'Bba Aab', 'Baa Aab'],
        ['Aaa Abb', 'Aab Abb', 'Abb Abb', 'Bbb Abb', 'Bba Abb', 'Baa Abb'],
        ['Aaa Bbb', 'Aab Bbb', 'Abb Bbb', 'Bbb Bbb', 'Bba Bbb', 'Baa Bbb'],
        ['Aaa Bba', 'Aab Bba', 'Abb Bba', 'Bbb Bba', 'Bba Bba', 'Baa Bba'],
        ['Aaa Baa', 'Aab Baa', 'Abb Baa', 'Bbb Baa', 'Bba Baa', 'Baa Baa']],
       dtype='<U7'),
 array([[ 1.        ,  0.90350095,  0.77839003,  0.2686965 ,  0.99446947,
          0.84250242],
        [ 0.90350095,  1.        ,  0.43421794,  0.65559232,  0.85349143,
          0.99207787],
        [ 0.77839003,  0.43421794,  1.        , -0.39554359,  0.84001846,
          0.31761454],
        [ 0.2686965 ,  0.65559232, -0.39554359,  1.        ,  0.16604679,
          0.74525953],
        [ 0.99446947,  0.85349143,  0.84001846,  0.16604679,  1.        ,
          0.7812662 ],
        [ 0.84250242,  0.99207787,  0